In [2]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=448ebd5d12355d2f50c66b4e82e7bd9845725658b7bae9910af273e06de0284e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
#disable warnings
import warnings
warnings.filterwarnings('ignore')
#common
import os
import traceback
from datetime import datetime
import random
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import pandas_ta as ta
import datetime as dt
from datetime import datetime
import psycopg2 as db
import scipy.stats
import plotly.graph_objects as go
import seaborn as sns

In [4]:
class StringConverter(dict):
    def __contains__(self, item):
        return True

    def __getitem__(self, item):
        return str

    def get(self, default=None):
        return str

In [7]:
# Reading users' Bank Account, Credit Card, Device and Orders data from csv
# Reading users' Bank Account, Credit Card, Device and Orders data from csv
bank_df  = pd.read_csv('/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/bank_accounts.csv', converters=StringConverter())
credit_df  = pd.read_csv('/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/credit_cards.csv', converters=StringConverter())
devices_df = pd.read_csv('/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/devices.csv', converters=StringConverter())
orders_df  = pd.read_csv('/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/orders.csv', converters=StringConverter())
# # chuyển dạng string cho userid trong đơn
# orders_df['buyer_code']=orders_df['buyer_userid'].astype(str)
# orders_df['seller_code']=orders_df['seller_userid'].astype(str)

In [8]:
# edit by Cuong
# Xóa trùng lặp trong bảng
bank_df = bank_df.drop_duplicates()
credit_df = credit_df.drop_duplicates()
devices_df = devices_df.drop_duplicates()
orders_df = orders_df.drop_duplicates()

In [9]:
# Get device, creditcard, account having more than once userid
df_user_device = devices_df.groupby('device').filter(lambda x: len(x) > 1)
df_user_creditcard = credit_df.groupby('credit_card').filter(lambda x: len(x) > 1)
df_user_account = bank_df.groupby('bank_account').filter(lambda x: len(x) > 1)

df_user_device = df_user_device.rename(columns={"device": "refitem"})
df_user_creditcard = df_user_creditcard.rename(columns={"credit_card": "refitem"})
df_user_account = df_user_account.rename(columns={"bank_account": "refitem"})
df_user_device['type']='device'
df_user_creditcard['type']='creditcard'
df_user_account['type']='account'

df_user_device=df_user_device.reset_index(drop=True)
df_user_creditcard=df_user_creditcard.reset_index(drop=True)
df_user_account=df_user_account.reset_index(drop=True)

In [10]:
# Build userset
frames = [df_user_device, df_user_creditcard, df_user_account]
df_user_relation = pd.concat(frames).reset_index(drop=True) #User relation
df_user_relation['device'] = df_user_relation.apply(lambda x: 1 if x.type == 'device' else 0, axis=1)
df_user_relation['account'] = df_user_relation.apply(lambda x: 1 if x.type == 'account' else 0, axis=1)
df_user_relation['creditcard'] = df_user_relation.apply(lambda x: 1 if x.type == 'creditcard' else 0, axis=1)

# chuyển về dữ liệu dạng chuỗi để build thành xâu ký các ID
df_user_relation['usercode']=df_user_relation['userid'].astype(str)

# Chuỗi user có cùng refitem
df_user_link = df_user_relation.groupby(['refitem', 'type'], as_index=False).agg({'usercode': ', '.join})
df_user_link = df_user_link.rename(columns={"usercode": "userset"})

# Combine với dữ liệu gốc để xác định mối quan hệ trực tiếp
df_user_relation = df_user_relation.merge(df_user_link, left_on=['refitem','type'], right_on = ['refitem','type'], how='left')

# Danh sách các user có quan hệ trực tiếp
df_user_set_direct = df_user_relation.groupby(['userid', 'usercode'], as_index=False).agg({'userset': ', '.join})

# chuyển dạng string cho userid trong đơn
orders_df['buyer_code']=orders_df['buyer_userid'].astype(str)
orders_df['seller_code']=orders_df['seller_userid'].astype(str)

# Các đơn hàng có tập buyer/seller cần quan tâm
# xuất hiện bên buyer & seller nằm trong tập userset
df_buyer_order = orders_df.merge(df_user_set_direct, left_on=['buyer_userid'], right_on = ['userid'], how='inner')
df_buyer_order['tag_direct'] = df_buyer_order.apply(lambda x: True if x.seller_code in x.userset else False, axis=1)

In [11]:
# Tìm đường đi ngắn nhất giữa hai usercode.
# Đường càng ngắn mối quan hệ càng gần, Số lượng đường đi giữa 02 user
# Tần suất xuất hiện order giữa 02 user trong order
df_buyer_order

,orderid,buyer_userid,seller_userid,buyer_code,seller_code,userid,usercode,userset,tag_direct
0,1953314712,131221930,72837912,131221930,72837912,131221930,131221930,"131221930, 215261568",False
1,1953453596,198638326,7314602,198638326,7314602,198638326,198638326,"198638326, 118947532",False
2,1953480060,130236220,77145726,130236220,77145726,130236220,130236220,"130236220, 130437218, 130236220, 130437218, 13...",False
3,1953487548,130236220,175676088,130236220,175676088,130236220,130236220,"130236220, 130437218, 130236220, 130437218, 13...",False
4,1953487546,130236220,55276200,130236220,55276200,130236220,130236220,"130236220, 130437218, 130236220, 130437218, 13...",False
...,...,...,...,...,...,...,...,...,...
54222,1955620920,47898518,54734168,47898518,54734168,47898518,47898518,"47898518, 13557450, 28123888, 13557450, 478985...",False
54223,1955772980,120814056,58851648,120814056,58851648,120814056,120814056,"120814056, 193791936, 120814056, 193791936, 79...",False
54224,1956206048,58672028,39287026,58672028,39287026,58672028,58672028,"58672028, 22308064, 58672028, 22308064, 189357992",False
54225,1956636054,14222102,70940136,14222102,70940136,14222102,14222102,"14222102, 29044192, 29044192, 14222102, 290441...",False


In [12]:
df_user_relation

,userid,refitem,type,device,account,creditcard,usercode,userset
0,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,device,1,0,0,33152428,"33152428, 8802740"
1,129669524,pdYhqPS3cmuPC3pm8HUVWD41lCxQJtT8Q5OmVmZ2ply=,device,1,0,0,129669524,"129669524, 27536218"
2,15190214,6xoWlzZrjCPxENoR9lXr09m+xNCxISXGiMJ6/GvNvnc=,device,1,0,0,15190214,"15190214, 15180794, 11628224"
3,133775686,Fo8aUEQzRpoZ7dQrprxb5Rl/hX81FoIfoSNk1lv9e7W=,device,1,0,0,133775686,"133775686, 260456"
4,29087736,KPaUrqhU+vW9/m+xp52tl/TFgJBE8e9kGKMxyVQSc2q=,device,1,0,0,29087736,"29087736, 6917000"
...,...,...,...,...,...,...,...,...
162422,17575470,2242100310,account,0,1,0,17575470,"43737260, 50074310, 42830430, 18629050, 249270..."
162423,15785630,2220022228894,account,0,1,0,15785630,"23225270, 15785630"
162424,11367690,391302003241821,account,0,1,0,11367690,"6983172, 11367690"
162425,23446396,884102022094820,account,0,1,0,23446396,"227980878, 23446396"


In [13]:
# Xuất file ra csv
df_user_relation.to_csv('/content/drive/Othercomputers/My Laptop/Shopee/user_relation.csv', index=False)